In [1]:
!pip install torch transformers spacy keybert PyPDF2
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 122.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import spacy
from keybert import KeyBERT
from PyPDF2 import PdfReader

In [3]:
device = 0 if torch.cuda.is_available() else -1
gpu_status = "GPU: " + torch.cuda.get_device_name(0) if device == 0 else "CPU"
print(f" Running on {gpu_status}")

 Running on GPU: Tesla T4


In [4]:
nlp = spacy.load("en_core_web_sm")
kw_model = KeyBERT()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive

In [5]:
!unzip /content/drive/MyDrive/multitask_fda_extractor.zip -d /content/drive/MyDrive/multitask_fda_extractor

Archive:  /content/drive/MyDrive/multitask_fda_extractor.zip
replace /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/special_tokens_map.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/tokenizer_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/labels.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/tokenizer.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/vocab.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
MODEL_PATH = "/content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/"

tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
model = BertForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=6)  # 6 heads
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/multitask_fda_extractor/content/multitask_fda_extractor/ and are newly initialized: ['bert.embeddings.LayerNorm.bias', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.word_embeddings.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [22]:
HEADS = ["Hardware", "Software Components", "AI Models", "Data Pipelines", "User Interface", "Integration"]
# Helper: run multitask predictions
def predict_heads(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits.squeeze()
    probs = torch.sigmoid(logits).cpu().numpy()
    return {HEADS[i]: float(probs[i]) for i in range(len(HEADS))}

# Helper: extract keywords from text
def extract_keywords(text, top_n=5):
    doc = nlp(text)
    chunks = list(set(chunk.text.lower() for chunk in doc.noun_chunks if len(chunk.text) > 3))
    if chunks:
        return chunks[:top_n]
    keybert_kw = kw_model.extract_keywords(text, top_n=top_n)
    return [kw for kw, _ in keybert_kw]

# Split PDF into text chunks
def extract_chunks_from_pdf(pdf_path, chunk_size=300):
    reader = PdfReader(pdf_path)
    full_text = " ".join([page.extract_text() for page in reader.pages if page.extract_text()])
    words = full_text.split()
    chunks = [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    print(f"Extracted {len(chunks)} chunks from PDF: {chunks}")
    return chunks

# Main pipeline: classify chunks + extract keywords with scores
def analyze_text_chunks(chunks, threshold=0.5):
    results = {head: {} for head in HEADS} # Change to dictionary to store scores
    for i, chunk in enumerate(chunks):
        head_probs = predict_heads(chunk)
        for head, prob in head_probs.items():
            if prob >= threshold:
                kws = extract_keywords(chunk)
                print(f"Chunk {i+1}: Extracted keywords for '{head}' (score: {prob:.4f}): {kws}")
                for kw in kws:
                    # Store keyword with its probability from the head prediction
                    # We'll use the chunk's probability for simplicity here,
                    # but ideally keyword relevance could also be considered.
                    results[head][kw] = prob # Store the probability
    # No need to remove duplicates and convert to list as dictionary handles uniqueness
    return results

In [27]:
# Instead of uploading a PDF, take text input directly
text_input = input("Enter text: ")

# Wrap the text input in a list to match the expected input of analyze_text_chunks
chunks = [text_input]

# Analyze the text chunks
output = analyze_text_chunks(chunks)
print(output)

Enter text: TAIMedImg DeepMets is a software application system intended for use in the contouring (segmentation) of brain magnetic resonance (MR) images. The device comprises an AI inference module and a DICOM Radiotherapy Structure Sets (RTSS, or RTSTRUCT) converter module. The AI inference module consists of image preprocessing, deep learning neural networks, and postprocessing components, and is intended to contour brain metastasis on the axial T1-weighted contrast-enhanced (T1WI+C) MR images. It utilizes deep learning neural networks to generate contours and annotations for the diagnosed brain metastases. The DICOM RTSS converter module converts the contours, annotations, along with metadata, into a standard DICOM-RTSTRUCT file, making it compatible with radiotherapy treatment planning systems.
Chunk 1: Extracted keywords for 'AI Models' (score: 0.5676): ['(mr) images', 'the device', 'a dicom radiotherapy structure sets', 'the axial t1-weighted contrast-enhanced (t1wi+c) mr images